In [ ]:
import numpy as np
from numpy.linalg import lstsq

# Splits the input matrix into non-overlapping segments of size s x s
def calculate_segments(X, s):
    M, N = X.shape  # Dimensions of the image
    M_s = M // s  # Number of vertical segments
    N_s = N // s  # Number of horizontal segments

    segments = []
    for v in range(M_s):
        for w in range(N_s):
            l1 = v * s
            l2 = w * s
            # Extracts an s x s segment from the input matrix
            segment = X[l1:l1+s, l2:l2+s]
            segments.append(segment)
    return segments 

# Computes the cumulative sum of the input segment along both axes
def cumulative_sum(segment):
    return np.cumsum(np.cumsum(segment, axis=1), axis=0)

# Removes the linear trend from the input segment
def detrend_surface(u):
    s = u.shape[0]  # Size of the segment
    X, Y = np.meshgrid(np.arange(1, s+1), np.arange(1, s+1))  # Grid coordinates
    X = X.flatten()
    Y = Y.flatten()
    Z = u.flatten()
    
    # Building the design matrix for the plane fitting
    A = np.vstack([X, Y, np.ones_like(X)]).T
    coeff, _, _, _ = lstsq(A, Z, rcond=None)  # Solves for the plane coefficients
    a, b, c = coeff
    
    # Calculates the linear trend (plane)
    u_trend = (a * X + b * Y + c).reshape(s, s)
    epsilon = u - u_trend  # Removes the trend from the original data
    
    return epsilon

# Calculates the fluctuation function for a detrended segment
def detrended_fluctuation_function(epsilon):
    s = epsilon.shape[0]  # Size of the segment
    # Computes the root mean square fluctuation
    F_vw_s = np.sqrt(np.sum(epsilon**2) / (s**2))
    return F_vw_s

# Computes the overall fluctuation function for the input image and scales
def overall_detrended_fluctuation(X, s, q):
    segments = calculate_segments(X, s)  # Splits the image into segments
    F_vw_s = []
    for segment in segments:
        u = cumulative_sum(segment)  # Computes the cumulative sum
        epsilon = detrend_surface(u)  # Detrends the segment
        F_vw_s.append(detrended_fluctuation_function(epsilon))  # Calculates fluctuation

    F_vw_s = np.array(F_vw_s)
    
    # Multifractal formalism for q
    if q == 0:
        F_q_s = np.exp(np.mean(np.log(F_vw_s)))  # Logarithmic mean for q = 0
    else:
        F_q_s = (np.mean(F_vw_s**q))**(1/q)  # Generalized mean for q != 0
    
    return F_q_s

# Performs the multifractal analysis for a range of scales and q values
def multifractal_analysis(X, s_values, q_values):
    F_q_s = {}
    for q in q_values:
        F_q_s[q] = []
        for s in s_values:
            F_q_s[q].append(overall_detrended_fluctuation(X, s, q))
        F_q_s[q] = np.array(F_q_s[q])
    
    return F_q_s

from skimage import io

# Loads the input image and processes it
image = io.imread('SARS2reference.png', as_gray=True)  # Reads the image in grayscale
io.imshow(image)

# Parameters for the multifractal analysis
q_values = np.arange(-10, 10, 1)  # Range of q values
s_values = np.arange(6, min(image.shape)//4, 2)  # Range of scales

# Computes the fluctuation functions
F_q_s = multifractal_analysis(image, s_values, q_values)

from scipy.optimize import curve_fit

# Linear function for fitting
def linear_fit(x, m, c):
    return m * x + c

# Calculates the scaling exponent h(q)
def calculate_hq(F_q_s, s_values, q_values):
    hq = {}
    for q in q_values:
        log_s = np.log(s_values)  # Logarithm of scales
        log_F_q_s = np.log(F_q_s[q])  # Logarithm of fluctuation function
        # Linear fit to find h(q)
        popt, _ = curve_fit(linear_fit, log_s, log_F_q_s)
        hq[q] = popt[0]  # Extracts the slope as h(q)
    return hq

# Calculates the mass exponent τ(q)
def calculate_tau_q(hq, q_values, D_f=2):
    tau_q = {q: q * hq[q] - D_f for q in q_values}  # τ(q) formula
    return tau_q

# Derives α(q) and f(α) from τ(q)
def calculate_alpha_f(tau_q, q_values):
    alpha_q = np.gradient(list(tau_q.values()), q_values)  # α(q) = dτ(q)/dq
    f_alpha = q_values * alpha_q - list(tau_q.values())  # f(α) formula
    return alpha_q, f_alpha




In [ ]:
# Calculates h(q), τ(q), α(q), and f(α) for the given fluctuation functions
hq = calculate_hq(F_q_s, s_values, q_values)
tau_q = calculate_tau_q(hq, q_values, D_f=2)
alpha_q, f_alpha = calculate_alpha_f(tau_q, q_values)